# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

Импортируем библиотеки

In [1]:
pip install optuna

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install fast_ml

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import optuna
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from fast_ml.model_development import train_valid_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from datetime import datetime

Загрузим данные и посмотрим, что у нас в датасете

In [4]:
try:
    data = pd.read_csv('/datasets/users_behavior.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/users_behavior.csv')

In [5]:
data.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [7]:
data.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


Т.к. уже в задании было сказано, что никаких предобработок делать не нужно, просто смотрим и убеждаемся, что в данных всё ОК

## Разбейте данные на выборки

Делим данные на обучающую (70%), валидационную (15%) и тестовую (15%) выборки

In [8]:
features_train, target_train, \
features_valid, target_valid, \
features_test, target_test = train_valid_test_split(data, 'is_ultra', train_size=0.7, valid_size=0.15, test_size=0.15)

При импорте train_valid_test_split произошла ошибка, по этому используем train_test_split дважды. Вначале делим данные на обучающую и тестовую выборку, потом из обучающей выборки забираем валидационную

Посмотрим размеры наших выборок:

In [9]:
print(f'Признаки обучающей выборки: {features_train.shape[0]}/{features_train.shape[1]},\n\
Признаки валидационной выборки: {features_valid.shape[0]}/{features_valid.shape[1]},\n\
Признаки тестовой выборки: {features_test.shape[0]}/{features_test.shape[1]},\n\
Целевой признак обучающей выборки: {target_train.shape[0]},\n\
Целевой признак валидационной выборки: {target_valid.shape[0]},\n\
Целевой признак тестовой выборки: {target_test.shape[0]}')

Признаки обучающей выборки: 2249/4,
Признаки валидационной выборки: 482/4,
Признаки тестовой выборки: 483/4,
Целевой признак обучающей выборки: 2249,
Целевой признак валидационной выборки: 482,
Целевой признак тестовой выборки: 483


## Исследуйте модели

#### GridSearchCV

Перебираем параметры решающего дерева с помощью модели GridSearchCV

In [10]:
start = datetime.now()

model_dtc = DecisionTreeClassifier(random_state=12345) # создаем модель решающего дерева
parametrs = {'criterion': ('gini', 'entropy', 'log_loss'),
             'max_depth': range (1, 14),
             'splitter': ('best', 'random'),
             'min_samples_split': range (2, 14, 2)} # список гиперпараметров для перебора модели решающего дерева
model_dtc = GridSearchCV(model_dtc, parametrs) # создаем модель для перебора гиперпарамтеров
model_dtc.fit(features_train, target_train) # обучаем и находим лучшие гиперпараметры

print(datetime.now() - start)

0:00:13.930530


In [11]:
predictions_dtc = model_dtc.predict(features_valid)
result_dtc = accuracy_score(target_valid, predictions_dtc)
print(result_dtc)

0.7800829875518672


In [12]:
print(f"Качество разбиения (criterion): {model_dtc.best_params_['criterion']};\n\
Стратегия разбиения в узлах (splitter): {model_dtc.best_params_['splitter']};\n\
Максимальная глубина дерева (max_depth): {model_dtc.best_params_['max_depth']};\n\
Минимальное кол-во объектов в узле для разбиения (min_samples_split): {model_dtc.best_params_['min_samples_split']}")

Качество разбиения (criterion): entropy;
Стратегия разбиения в узлах (splitter): best;
Максимальная глубина дерева (max_depth): 6;
Минимальное кол-во объектов в узле для разбиения (min_samples_split): 10


#### optuna

Попробую ещё и вторым способом. Первый раз использовал optuna на случайном лесу, показалось, что параметры подбираются быстрее

In [13]:
def find_params_dtc(trial: optuna.Trial):
    params = {'criterion': trial.suggest_categorical('criterion', ['gini', 'entropy']),
              'max_depth': trial.suggest_int('max_depth', 1, 14),
              'splitter': trial.suggest_categorical('splitter', ['best', 'random']),
              'min_samples_split': trial.suggest_int('min_samples_split', 2, 14, 2)}

    model_dtc_opt = DecisionTreeClassifier(random_state=12345)
    model_dtc_opt.fit(features_train, target_train)
    predictions_dtc_opt = model_dtc_opt.predict(features_valid)
    result_dtc_opt = accuracy_score(target_valid, predictions_dtc_opt)
    return result_dtc_opt

start = datetime.now()
study = optuna.create_study(direction='minimize')
# study = optuna.create_study()
study.optimize(find_params_dtc, n_trials=100)

print(datetime.now() - start)

[I 2023-06-03 14:45:44,807] A new study created in memory with name: no-name-dbc97bf0-7519-473d-a46c-47ad8847ea01
[I 2023-06-03 14:45:44,823] Trial 0 finished with value: 0.6950207468879668 and parameters: {'criterion': 'gini', 'max_depth': 7, 'splitter': 'random', 'min_samples_split': 2}. Best is trial 0 with value: 0.6950207468879668.
[I 2023-06-03 14:45:44,837] Trial 1 finished with value: 0.6950207468879668 and parameters: {'criterion': 'entropy', 'max_depth': 13, 'splitter': 'best', 'min_samples_split': 8}. Best is trial 0 with value: 0.6950207468879668.
[I 2023-06-03 14:45:44,878] Trial 2 finished with value: 0.6950207468879668 and parameters: {'criterion': 'gini', 'max_depth': 3, 'splitter': 'random', 'min_samples_split': 14}. Best is trial 0 with value: 0.6950207468879668.
[I 2023-06-03 14:45:44,901] Trial 3 finished with value: 0.6950207468879668 and parameters: {'criterion': 'entropy', 'max_depth': 8, 'splitter': 'best', 'min_samples_split': 8}. Best is trial 0 with value: 0.

0:00:03.834397


In [14]:
model_dtc_opt = DecisionTreeClassifier(**study.best_params, random_state=12345)
model_dtc_opt.fit(features_train, target_train)
predictions_dtc_opt = model_dtc_opt.predict(features_valid)
result_dtc_opt = accuracy_score(target_valid, predictions_dtc_opt)
print(result_dtc_opt)

0.7614107883817427


In [15]:
print(f"Качество разбиения (criterion): {study.best_params['criterion']};\n\
Стратегия разбиения в узлах  (splitter): {study.best_params['splitter']};\n\
Максимальная глубина деревьев (max_depth): {study.best_params['max_depth']};\n\
Минимальное кол-во объектов в узле для разбиения (min_samples_split): {study.best_params['min_samples_split']}")

Качество разбиения (criterion): gini;
Стратегия разбиения в узлах  (splitter): random;
Максимальная глубина деревьев (max_depth): 7;
Минимальное кол-во объектов в узле для разбиения (min_samples_split): 2


Результаты получились немного разными как и по значениям гиперпараметров, так и по скорости. Я не стал делать поиск более 4 парамтеров, т.к. это начинает занимать много время. Больше параметров было у случайного леса... там поиск затянулся аж на 1 мин. :( прошу прощения за это

In [16]:
print(sklearn.__version__)

0.24.1


Мне пришлось убрать значение "log_loss" из гиперпараметра criterion тк в какой-то момент начала вылетать ошибка

### случайный лес

Перебираем параметры решающего дерева с помощью фреймворка optuna

In [17]:
def find_params_rfc(trial: optuna.Trial):
    params = {'n_estimators': trial.suggest_int('n_estimators', 10, 101, 10),
              'criterion': trial.suggest_categorical('criterion', ['gini', 'entropy']),
              'max_depth': trial.suggest_int('max_depth', 1, 15, 1),
              'min_samples_split': trial.suggest_int('min_samples_split', 2, 12, 2),
              'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 11)}

    model_rfc_opt = RandomForestClassifier(random_state=12345)
    model_rfc_opt.fit(features_train, target_train)
    predictions_rfc_opt = model_rfc_opt.predict(features_valid)
    result_rfc_opt = accuracy_score(target_valid, predictions_rfc_opt)
    return result_rfc_opt

start = datetime.now()
# study = optuna.create_study(direction='minimize')
study = optuna.create_study()
study.optimize(find_params_rfc, n_trials=100)

print(datetime.now() - start)

[I 2023-06-03 14:45:48,688] A new study created in memory with name: no-name-6fe485df-0b5d-4866-a81a-761b3c8d81cb
[I 2023-06-03 14:45:49,247] Trial 0 finished with value: 0.7883817427385892 and parameters: {'n_estimators': 70, 'criterion': 'entropy', 'max_depth': 4, 'min_samples_split': 2, 'min_samples_leaf': 4}. Best is trial 0 with value: 0.7883817427385892.
[I 2023-06-03 14:45:49,772] Trial 1 finished with value: 0.7883817427385892 and parameters: {'n_estimators': 80, 'criterion': 'gini', 'max_depth': 4, 'min_samples_split': 4, 'min_samples_leaf': 1}. Best is trial 0 with value: 0.7883817427385892.
[I 2023-06-03 14:45:50,316] Trial 2 finished with value: 0.7883817427385892 and parameters: {'n_estimators': 70, 'criterion': 'gini', 'max_depth': 2, 'min_samples_split': 4, 'min_samples_leaf': 8}. Best is trial 0 with value: 0.7883817427385892.
[I 2023-06-03 14:45:50,838] Trial 3 finished with value: 0.7883817427385892 and parameters: {'n_estimators': 100, 'criterion': 'entropy', 'max_de

0:00:59.636544


In [18]:
model_rfc_opt = RandomForestClassifier(**study.best_params, random_state=12345)
model_rfc_opt.fit(features_train, target_train)
predictions_rfc_opt = model_rfc_opt.predict(features_valid)
result_rfc_opt = accuracy_score(target_valid, predictions_rfc_opt)
print(result_rfc_opt)

0.7821576763485477


In [19]:
print(f"Число деревьев (n_estimators): {study.best_params['n_estimators']};\n\
Качество разбиения (criterion): {study.best_params['criterion']};\n\
Максимальная глубина деревьев (max_depth): {study.best_params['max_depth']};\n\
Минимальное кол-во объектов в узле для разбиения (min_samples_split): {study.best_params['min_samples_split']};\n\
Минимальное кол-во объектов в листьях (min_samples_leaf): {study.best_params['min_samples_leaf']}")

Число деревьев (n_estimators): 70;
Качество разбиения (criterion): entropy;
Максимальная глубина деревьев (max_depth): 4;
Минимальное кол-во объектов в узле для разбиения (min_samples_split): 2;
Минимальное кол-во объектов в листьях (min_samples_leaf): 4


### логистическая регрессия

Перебираем параметры решающего дерева с помощью модели GridSearchCV.  
Мне этот метод понравился только тем, что он из библиотеки sclear, хотя где-то в статье я прочитал, что optuna начал набирать популярности, я всё равно пока за GridSearchCV. Но объективно я разницы не понимаю

In [20]:
start = datetime.now()

model_lr = LogisticRegression(random_state=12345) # создаем модель логической регрессии
parametrs = {'penalty': ('l1', 'l2', 'elasticnet', None),
             'solver': ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'],
             'C': (-5, 5, 75)} # список гиперпараметров для перебора модели решающего дерева
model_lr = GridSearchCV(model_lr, parametrs) # создаем модель для перебора гиперпарамтеров
model_lr.fit(features_train, target_train) # обучаем и находим лучшие гиперпараметры

print(datetime.now() - start)

0:00:04.167889


In [21]:
predictions_lr = model_lr.predict(features_valid)
result_lr = accuracy_score(target_valid, predictions_lr)
print(result_lr)

0.7282157676348547


In [22]:
print(f"категория штрафа за ошибки (penalty): {model_lr.best_params_['penalty']};\n\
Алгоритм решения (solver): {model_lr.best_params_['solver']};\n\
Параметр неверной классификации (C): {model_lr.best_params_['C']}")

категория штрафа за ошибки (penalty): l2;
Алгоритм решения (solver): newton-cg;
Параметр неверной классификации (C): 5


In [23]:
print(f"Посмотрим результаты обученных моделей на валидационной выборке и выберим лучший:\n\
решающее дерево: {result_dtc} / {result_dtc_opt};\n\
случайный лес: {result_rfc_opt};\n\
логистическая регрессия: {result_lr}")

Посмотрим результаты обученных моделей на валидационной выборке и выберим лучший:
решающее дерево: 0.7800829875518672 / 0.7614107883817427;
случайный лес: 0.7821576763485477;
логистическая регрессия: 0.7282157676348547


После использования автоматизации при подборе гиперпараметров произошли изменения в данных, но  
всё равно ↓

**Выбираем модель "Случайный лес"**

## Проверьте модель на тестовой выборке

Смотрим модель решающего дерева на тестовой выборке

In [24]:
predictions_rfc2 = model_rfc_opt.predict(features_test) # получите предсказания модели
result_rfc2 = accuracy_score(target_test, predictions_rfc2)
result_rfc2

0.8033126293995859

На тестовой выборке модель показала себя почти также, как при валидации, это говорит о том. что модель и гиперпараметры подобраны хорошо

## Общий вывод

Мы посмотрели тры разные модели для задачи классификации:  
 - Решающее дерево
 - Случайный лес
 - Логическая регрессия  
 
Случайный лес с глубиной 17 показал лучшее качество (accuracy) - 0.78 на валидационной выборке и 0.778 на тестовой выборке.[В обучении](https://practicum.yandex.ru/trainer/data-scientist/lesson/76a8579c-f459-4622-b24a-4472f2e46add/#:~:text=%D0%9E%D0%B1%D1%8A%D0%B5%D0%B4%D0%B8%D0%BD%D0%B8%D0%BC%20%D0%BA%D1%80%D0%B8%D1%82%D0%B5%D1%80%D0%B8%D0%B8%20%D0%B2-,%D1%82%D0%B0%D0%B1%D0%BB%D0%B8%D1%86%D1%83%3A,-Name) приводилась сравнительная таблица "качество/скорость моелей", вопреки таблицы Логическая регрессия показала худший результат